<a href="https://colab.research.google.com/github/GhazalehKeyvani/SLM/blob/main/personal_sml.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install --upgrade --quiet langchain langchain-community langchain-groq langchain-experimental neo4j
print('installed')

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 13.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 61.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 209.2/209.2 kB 16.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 312.3/312.3 kB 18.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 121.9/121.9 kB 8.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 3.1 MB/s eta 0:00:00
installed


In [ ]:
NEO4J_URI="neo4j+s://924bf097.databases.neo4j.io"
NEO4J_USERNAME="neo4j"
NEO4J_PASSWORD="F7AzauN30ypr91Lt_83EB0QsZ17_oE_fwYBlGSyh7ak"

In [ ]:
import os
os.environ["NEO4J_URI"]=NEO4J_URI
os.environ["NEO4J_USERNAME"]=NEO4J_USERNAME
os.environ["NEO4J_PASSWORD"]=NEO4J_PASSWORD

In [ ]:
from langchain_community.graphs import Neo4jGraph
graph = Neo4jGraph(
    url=NEO4J_URI,  # Use the variable directly instead of the string 'NEO4J_URI'
    username=NEO4J_USERNAME,  # Use the variable directly instead of the string 'NEO4J_USERNAME'
    password=NEO4J_PASSWORD,  # Use the variable directly instead of the string 'NEO4J_PASSWORD'
)

<ipython-input-6-ecc03a609189>:2: LangChainDeprecationWarning: The class `Neo4jGraph` was deprecated in LangChain 0.3.8 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-neo4j package and should be used instead. To use it run `pip install -U :class:`~langchain-neo4j` and import as `from :class:`~langchain_neo4j import Neo4jGraph``.
  graph = Neo4jGraph(


In [ ]:
from neo4j import GraphDatabase

# اتصال به پایگاه داده Neo4j
driver =GraphDatabase.driver(NEO4J_URI, auth=("neo4j", NEO4J_PASSWORD))

def save_to_neo4j(question, answer):
    with driver.session() as session:
        session.run("CREATE (q:Question {text: $question})-[:HAS_ANSWER]->(a:Answer {text: $answer})",
                    question=question, answer=answer)

# مثال استفاده از تابع
save_to_neo4j("سلام", "سلام، چطور می‌تونم کمک‌تون کنم؟")

In [ ]:
from langchain.embeddings import SentenceTransformerEmbeddings
from langchain.text_splitter import CharacterTextSplitter
from langchain_community.document_loaders import DirectoryLoader, PyPDFLoader
from langchain_core.documents import Document

# تعریف مدل embedding
embedding_provider = SentenceTransformerEmbeddings(model_name="all-mpnet-base-v2")

# تابع برای ایجاد گراف دانش
def create_knowledge_graph(chunks):
    for chunk in chunks:
        filename = os.path.basename(chunk.metadata["source"])
        chunk_id = f'{filename}.{chunk.metadata["page"]}'
        print("Processing -", chunk_id)

        # ایجاد embedding برای chunk
        chunk_embedding = embedding_provider.embed_query(chunk.page_content)

        # اضافه کردن Document و Chunk به گراف
        properties = {
            "filename": filename,
            "chunk_id": chunk_id,
            "text": chunk.page_content,
            "embedding": chunk_embedding
        }

        graph.query("""
            MERGE (d:Document {id: $filename})
            MERGE (c:Chunk {id: $chunk_id})
            SET c.text = $text
            MERGE (d)<-[:PART_OF]-(c)
            WITH c
            CALL db.create.setNodeVectorProperty(c, 'textEmbedding', $embedding)
            """,
            properties
        )

        # ایجاد موجودیت‌ها و روابط از chunk
        graph_docs = doc_transformer.convert_to_graph_documents([chunk])

        # نگاشت موجودیت‌ها در گراف به chunk node
        for graph_doc in graph_docs:
            chunk_node = Node(
                id=chunk_id,
                type="Chunk"
            )

            for node in graph_doc.nodes:
                graph_doc.relationships.append(
                    Relationship(
                        source=chunk_node,
                        target=node,
                        type="HAS_ENTITY"
                    )
                )

        # اضافه کردن graph documents به گراف
        graph.add_graph_documents(graph_docs)

    # ایجاد ایندکس برداری
    graph.query("""
        CREATE VECTOR INDEX `chunkVector`
        IF NOT EXISTS
        FOR (c: Chunk) ON (c.textEmbedding)
        OPTIONS {indexConfig: {
        `vector.dimensions`: 1536,
        `vector.similarity_function`: 'cosine'
        }};""")

<ipython-input-8-fa011fe6bd49>:7: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embedding_provider = SentenceTransformerEmbeddings(model_name="all-mpnet-base-v2")
/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or dataset

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.6k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/363 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

1_Pooling%2Fconfig.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [ ]:

def retrieve_relevant_chunks(question):
    # ایجاد embedding برای سوال
    question_embedding = embedding_provider.embed_query(question)

    # جستجو در ایندکس برداری برای یافتن chunk‌های مرتبط
    query = """
        WITH $question_embedding AS question_embedding
        CALL gds.graph.project(
    'myGraph',
    'Chunk',
    {
        PART_OF: {
            orientation: 'UNDIRECTED'
        }
    },
    {
        nodeProperties: ['textEmbedding']
    }
);

        MATCH (c:Chunk)
        WHERE gds.similarity.cosine(c.textEmbedding, question_embedding) > 0.7
        RETURN c
        ORDER BY gds.similarity.cosine(c.textEmbedding, question_embedding) DESC
        LIMIT 5
    """
    #print(query)
    result = graph.query(query, {"question_embedding": question_embedding})
    relevant_chunks = [record["c"] for record in result]

    return relevant_chunks

In [ ]:
from sentence_transformers import SentenceTransformer, util
import torch

# Load the sentence-transformer model
model = SentenceTransformer('all-MiniLM-L6-v2')

# Example question and contexts
question = "What is the capital of France?"
contexts = ["Paris is the capital of France.", "Berlin is the capital of Germany.", "Madrid is the capital of Spain."]

# Encode the question and contexts
question_embedding = model.encode(question, convert_to_tensor=True)
context_embeddings = model.encode(contexts, convert_to_tensor=True)

# Compute cosine similarities
cosine_similarities = util.pytorch_cos_sim(question_embedding, context_embeddings)

# Convert similarities to list and print them
similarities = cosine_similarities.squeeze().tolist()
print(similarities)

# Find the context with the highest similarity
most_similar_context = contexts[torch.argmax(cosine_similarities)]
print(f"Most similar context: {most_similar_context}")

[0.8560698628425598, 0.4033600986003876, 0.4169125556945801]
Most similar context: Paris is the capital of France.


In [ ]:
pip install gradio

In [ ]:
import gradio as gr
from neo4j import GraphDatabase

# اتصال به Neo4j
driver = GraphDatabase.driver(NEO4J_URI, auth=("neo4j",NEO4J_PASSWORD ))

# تابع برای ذخیره‌سازی سوال و جواب در Neo4j
def save_to_neo4j(question, answer):
    with driver.session() as session:
        session.run("CREATE (q:Question {text: $question})-[:HAS_ANSWER]->(a:Answer {text: $answer})",
                    question=question, answer=answer)

# تابع برای ایجاد پاسخ با استفاده از RAG
def generate_response_with_rag(question):
    # اینجا می‌توانید از کدهای RAG خود استفاده کنید
    return f"پاسخ به سوال: {question}"

# تابع اصلی برای رابط کاربری
def chatbot_interface(question):
    if question:
        # تولید پاسخ با استفاده از RAG
        answer = generate_response_with_rag(question)

        # ذخیره‌سازی سوال و جواب در Neo4j
        save_to_neo4j(question, answer)

        return answer
    else:
        return "لطفاً یک سوال وارد کنید."

# ایجاد رابط کاربری با Gradio
interface = gr.Interface(
    fn=chatbot_interface,
    inputs=gr.Textbox(lines=2, placeholder="سوال خود را اینجا وارد کنید..."),
    outputs="text",
    title="سیستم پرسش و پاسخ هوشمند",
    description="لطفاً سوال خود را وارد کنید و پاسخ هوشمند دریافت کنید."
)

# اجرای رابط کاربری
interface.launch()

Running Gradio in a Colab notebook requires sharing enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://3d85e8735f50b101e0.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
